In [7]:
import pandas as pd

In [8]:
# Importacion de Plotly como libreria de DataViz
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [9]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [10]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
blueTeam["Team"] = "Blue"
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
redTeam["Team"] = "Red"
data = pd.concat([blueTeam,redTeam])

In [11]:
# Reemplazar los valores de la columna "Wins"
data['WinTags'] = data['Wins'].replace({0: 'Loss', 1: 'Win'})

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}
data.drop(columns=["Unnamed: 0"], inplace=True)

### Encoding:
- Para el encoding, necesitas convertir las variables categóricas en formatos numéricos para que los algoritmos de Machine Learning puedan procesarlos. La columna 'Team' es la única columna categórica en tu conjunto de datos. Puedes aplicar encoding one-hot a esta columna para convertirla en características binarias (0 y 1) para cada equipo.

In [12]:
data = pd.get_dummies(data, columns=['Team'])

### Feature Engineering:
En este paso, puedes crear nuevas características o transformar las existentes para mejorar el rendimiento del modelo. Piensa en qué características podrían ser relevantes en el contexto de una partida de League of Legends a los 10 minutos. Algunas ideas podrían ser:
- Ratio de Kills/Deaths (puede indicar el rendimiento de un jugador).
- Ratio de WardsPlaced/WardsDestroyed (puede indicar la eficacia en el control del mapa).
- Ratio de GoldDiff/ExperienceDiff (puede reflejar el equilibrio entre los equipos).

### Separación de Datos:
- Divide tu conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba para evaluar el rendimiento de tu modelo. Puedes usar, por ejemplo, un 80% de los datos para entrenamiento y un 20% para prueba.

In [15]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['WinTags'])  # X serán las características, excluyendo la columna objetivo (Wins)
y = data['WinTags']  # y será la variable objetivo (Wins)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Entrenamiento de un Modelo de Machine Learning:
- Elige un algoritmo de clasificación que se adapte a tu problema. Puedes probar con varios algoritmos y comparar su rendimiento. Algunos ejemplos son Logistic Regression, Random Forest, Support Vector Machines, o Gradient Boosting.

In [33]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

### Evaluación del Modelo:
- Evalúa el modelo en el conjunto de prueba utilizando métricas apropiadas, como precisión, recall, F1-score o matriz de confusión. Ajusta hiperparámetros si es necesario.

In [34]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(report)


Accuracy: 1.0
              precision    recall  f1-score   support

        Loss       1.00      1.00      1.00      1937
         Win       1.00      1.00      1.00      2015

    accuracy                           1.00      3952
   macro avg       1.00      1.00      1.00      3952
weighted avg       1.00      1.00      1.00      3952



### Ajuste y Optimización del Modelo:
- Si el rendimiento no es satisfactorio, puedes ajustar hiperparámetros, probar diferentes algoritmos o realizar ingeniería de características adicional para mejorar el modelo.

##### Ajuste de Hiperparámetros:
 - Cada algoritmo tiene hiperparámetros que pueden ajustarse para mejorar el rendimiento del modelo. Por ejemplo, en el caso de RandomForestClassifier, puedes ajustar el número de árboles, la profundidad máxima de los árboles, etc.

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

 - Este código utiliza la validación cruzada para probar diferentes combinaciones de hiperparámetros y encontrar los mejores valores.

##### Feature Importance:

- Utiliza la propiedad feature_importances_ de tu modelo para entender qué características son más importantes en la toma de decisiones del modelo. Puedes eliminar características menos importantes para simplificar el modelo.

In [20]:
feature_importances = best_model.feature_importances_

##### Validación Cruzada:

Realiza la validación cruzada para obtener una evaluación más robusta del rendimiento de tu modelo en diferentes conjuntos de datos.

In [22]:
from sklearn.model_selection import cross_val_score
import numpy as np

cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
average_cv_accuracy = np.mean(cv_scores)


##### Curva de Aprendizaje:

Plotea la curva de aprendizaje para entender si tu modelo se beneficiaría de más datos de entrenamiento o si está sobreajustado.

In [25]:
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, X, y, cv=5):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv)

    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=train_sizes,
        y=train_scores_mean,
        mode='lines',
        name='Training Score',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=train_sizes,
        y=test_scores_mean,
        mode='lines',
        name='Cross-validation Score',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title='Curva de Aprendizaje',
        xaxis_title='Tamaño del Conjunto de Entrenamiento',
        yaxis_title='Puntuación de Exactitud',
        legend=dict(x=0.02, y=0.95),
        margin=dict(l=0, r=0, t=30, b=0)
    )

    fig.show()

# Utiliza la función con tu mejor modelo y conjuntos de entrenamiento
plot_learning_curve(best_model, X_train, y_train)

##### Ensemble Methods:

Puedes probar métodos de conjunto (ensemble methods) como Bagging, Boosting, o Stacking para mejorar el rendimiento del modelo.

In [24]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(base_estimator=best_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=50,
                                                        random_state=42),
                  random_state=42)

### Conclusión:
A pesar de que tanto el modelo base como el mejor modelo lograron una eficacia del 100%, estamos actualmente en la fase de evaluación de usabilidad para detectar posibles casos de sobreajuste. Se observa claramente una señal de sobreajuste debido a la presencia de registros anómalos, comúnmente conocidos como "Remontadas épicas". A pesar de la consistencia del equipo perdedor durante los primeros 30 minutos, hay instancias donde el equipo, inicialmente en desventaja, muestra una notable persistencia, revirtiendo el curso de la partida y siendo catalogado por el modelo como perdedor. Este fenómeno desafía la lógica del modelo al reconocer la victoria del equipo que, según su evaluación, debería haber perdido.

### Recursos
- [Use Random Forest to predict on Walmart Sales](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-walmart-sales-ae6ebadb569b)
- [Isolation Forest](https://medium.com/@corymaklin/isolation-forest-799fceacdda4)
- [ML Dataset — How To Prepare a Good One?](https://betterprogramming.pub/ml-dataset-how-to-prepare-a-good-one-7d92ce1d45e5)
- [Use Random Forest to predict on a multiclass imbalance problem](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-a-multiclass-imbalance-problem-fd0259098ac4)